# Bienvenue sur le fichier de mise en forme

Avant de lancer ce script :
1. Placez le fichier "compte_mandat.xls" que vous souhaitez formater dans le même dossier que ce notebook.
2. Assurez-vous bien que ce fichier s'appel "compte_mandat.xls"
3. S'il existe, fermez le fichier excel "compte_mandat_clean.xls" autrement, le fait qu'il soit ouvert peut empêcher le fonctionnement de ce script

Après vérification, vous pouvez lancer le script :
1. Dans la barre au dessus, cliquez sur le bouton <button class="btn btn-default" title="redémarrer le noyau, et ré-exécuter tout le notebook (sans confirmation)"><i class="fa-forward fa"></i></button>
2. Les cellules de ce notebook seront jouées les unes après les autres.
3. La toute dernière cellule produira le fichier excel correctement formatté, appelé "compte_mandat_clean.xls"

<i class='fa fa-warning'></i> Si des erreurs apparaissent au long de ce script, alors soit :
* Le fichier Excel source est manquant
* Le fichier Excel source est incorrectement nommé
* Le fichier Excel destination est ouvert, ce qui empêche l'export d'un nouveau fichier Excel

In [12]:
# Importation de librairies
import sys
sys.tracebacklimit = 0
import re
import glob
import pandas as pd
from tabulate import tabulate
from datetime import datetime, timedelta, date
import numpy as np
import os
from dateutil.relativedelta import relativedelta
import time
from pathlib import Path
from openpyxl import load_workbook
pd.set_option('display.max_colwidth', None)
# pd.set_option('display.width', 1000)

In [16]:
# On récupère le fichier dans le répertoire courant.

li = []
error_log=[]
filenames=[]
filename=glob.glob("compte_mandat.xls")[0] 
    
df=pd.read_excel(filename, engine="xlrd", header = None)
print("\nNe pas tenir compte du message ci-dessus ci-dessus.")


*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'

Ne pas tenir compte du message ci-dessus ci-dessus.


In [ ]:
# On élimine l'ensemble des colonnes qui ne servent à rien
col_to_drop=[]
df_height,_=df.shape

intitule=df[14].iloc[3]
print(intitule)

for col in df.columns:
    count = df[col].isna().sum()
    if (count/df_height)>0.95:
        col_to_drop.append(col)
# On drop les premières lignes qui ne servent à rien
df_drop=df.drop(col_to_drop, axis=1)
df_drop = df_drop.iloc[4: , :]

41100027 MORANCE SERGE
---
        0           3     6         7    9                               11  \
4   3299.0  08/01/2015   3.0  512001.0   PR         REGLT. PLANETE PUBLIQUE   
5   3346.0  12/01/2015  21.0  512001.0   VI             DENIS: APPEL 1T2015   
6   3392.0  19/01/2015  27.0  512001.0   OD   TP: TVA A PAYER DECEMBRE 2014   
7   3598.0  20/01/2015   2.0  512001.0   VI          REGLEMENT PAR VIREMENT   
8      NaN         NaN   NaN       NaN  NaN                             NaN   
9   3711.0  08/02/2015   3.0  512001.0   PR         REGLT. PLANETE PUBLIQUE   
10  3834.0  11/02/2015  22.0  512001.0   VI  DENIS: ACT 12 & 13 TVX AG 2013   
11  3879.0  13/02/2015  13.0  512001.0   OD           TP: TAXE BUREAUX 2015   
12  3914.0  16/02/2015   2.0  512001.0   VI          REGLEMENT PAR VIREMENT   
13  3888.0  19/02/2015  27.0  512001.0   OD    TP: TVA A PAYER JANVIER 2015   
14     NaN         NaN   NaN       NaN  NaN                             NaN   
15  4014.0  08/03/2015   

In [18]:
# Dans les colonnes 1 (Pièce) et 14 (Sous-Total), s'il y a une cellule vide, je la renseine à partir de la prochaine valeur non-vide situé en dessous d'elle.
# df_drop[1].bfill(inplace=True)
# df_drop[14].bfill(inplace=True)

# On créer un alias pour éviter de perdre la donnée déjà obtenue et travailler sur une copie
df_alias=df_drop

# On supprime toutes les lignes dont la colonne 0 (Date) est nulle
# df_alias = df.drop(df[df[0].isnull()].index)

# On remplace le mot "Pièce" par "" (rien) pour éviter que le tableau soit trop chargé inutillement
# df_alias.replace("Pièce : ","",regex=True,inplace=True)
df_alias.replace(np.nan,"",regex=True,inplace=True)

# On renomme les colonnes pour avoir des libellés clairs
df_alias.rename(columns={0: "Pièce"},inplace=True)
df_alias.rename(columns={3: "Date"},inplace=True)
df_alias.rename(columns={6: "C.A"},inplace=True)
df_alias.rename(columns={7: "Compte"},inplace=True)
df_alias.rename(columns={9: "Jal"},inplace=True)
df_alias.rename(columns={11: "Intitulé"},inplace=True)
df_alias.rename(columns={13: ""},inplace=True)
df_alias.rename(columns={17: "N° Chèque"},inplace=True)
df_alias.rename(columns={21: "Débit"},inplace=True)
df_alias.rename(columns={25: "Crédit"},inplace=True)
df_alias.rename(columns={27: "Solde"},inplace=True)

       Pièce        Date   C.A      Compte Jal                       Intitulé  \
4     3299.0  08/01/2015   3.0    512001.0  PR        REGLT. PLANETE PUBLIQUE   
5     3346.0  12/01/2015  21.0    512001.0  VI            DENIS: APPEL 1T2015   
6     3392.0  19/01/2015  27.0    512001.0  OD  TP: TVA A PAYER DECEMBRE 2014   
7     3598.0  20/01/2015   2.0    512001.0  VI         REGLEMENT PAR VIREMENT   
8                                                                               
..       ...         ...   ...         ...  ..                            ...   
394  10687.0  16/10/2019  13.0    512001.0  OD         TP: TAXE FONCIERE 2019   
395                                                                             
396  11064.0  08/11/2019  13.0  41100273.0  OD        RBST TAXE FONCIERE 2019   
397                                                                             
398                                                                             

                     N° Chè

In [5]:
# result_df = df_alias.drop_duplicates(subset=['Libellé'], keep='first')
# sort_df=result_df.sort_values(by=['Libellé'])

In [19]:
filename=filename.split(".")[0]
print(filename)
# On exporte en un fichier excel propre
# Si erreur, vérifiez que le fichier excel n'est pas déjà ouvert
df_alias.to_excel(filename+"_"+intitule+"_"+'_clean.xlsx', engine='xlsxwriter',index=False)     

compte_mandat
Export réussi !
Le fichier suivant a été créé :
- compte_mandat_41100027 MORANCE SERGE__clean.xlsx
